In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
import joblib
from IPython.display import clear_output 
from google.colab import files

2024-08-24 21:46:03.352716: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-24 21:46:03.352886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 21:46:03.513835: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from google.colab import files
files.upload()
clear_output(wait=True)


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mlg-ulb/creditcardfraud
!unzip -o creditcardfraud.zip
!rm -rf creditcardfraud.zip

In [3]:
df = pd.read_csv('creditcard.csv')

X = df.drop(columns=['Class'])
y = df['Class']



In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1, shuffle=True)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test, X_val , y_test, y_val = train_test_split(X_test, y_test, train_size=0.5, random_state=1, shuffle=True)

In [ ]:
joblib.dump(scaler, 'scaler.pkl')

In [60]:
y_train = y_train.values
y_test = y_test.values
y_val = y_val.values

In [6]:
# TPU configuration
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
except ValueError:
    print("TPU not found, using CPU/GPU instead")
    strategy = tf.distribute.get_strategy()

TPU not found, using CPU/GPU instead


In [38]:
with strategy.scope():
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dropout(0.6),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  # Keep the loss suitable for binary classification
        metrics=[
            tf.keras.metrics.Recall(),  # Prioritize recall
            tf.keras.metrics.Precision(),  # Optionally track precision as well
            'accuracy'  # Still track accuracy for reference
        ]
    )


early_stopping = EarlyStopping(monitor='val_recall', patience=10, mode='max', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# Class weights to priotrize class 1
class_weight = {0: 1., 1: 8}

In [62]:
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val),
                    verbose=1,
                    callbacks=[early_stopping, reduce_lr],
                    class_weight=class_weight)
     

Epoch 1/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9993 - loss: 0.0139 - precision_2: 0.8098 - recall_2: 0.8214 - val_accuracy: 0.9992 - val_loss: 0.0035 - val_precision_2: 0.7660 - val_recall_2: 0.7660 - learning_rate: 2.5000e-04
Epoch 2/20
  92/7121 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9988 - loss: 0.0046 - precision_2: 0.3107 - recall_2: 0.5217          

/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_recall` which is not available. Available metrics are: accuracy,loss,precision_2,recall_2,val_accuracy,val_loss,val_precision_2,val_recall_2
  current = self.get_monitor_value(logs)


7121/7121 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9993 - loss: 0.0143 - precision_2: 0.7967 - recall_2: 0.8122 - val_accuracy: 0.9992 - val_loss: 0.0041 - val_precision_2: 0.7500 - val_recall_2: 0.7660 - learning_rate: 2.5000e-04
Epoch 3/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9994 - loss: 0.0129 - precision_2: 0.8024 - recall_2: 0.8391 - val_accuracy: 0.9992 - val_loss: 0.0037 - val_precision_2: 0.7500 - val_recall_2: 0.7660 - learning_rate: 2.5000e-04
Epoch 4/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9992 - loss: 0.0130 - precision_2: 0.7630 - recall_2: 0.8380 - val_accuracy: 0.9992 - val_loss: 0.0037 - val_precision_2: 0.7609 - val_recall_2: 0.7447 - learning_rate: 2.5000e-04
Epoch 5/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9994 - loss: 0.0103 - precision_2: 0.8040 - recall_2: 0.8625 - val_accuracy: 0.9992 - val_loss: 0.0044 - val_precision_2: 0.7500 - val_recall_2: 0.7660 - learning_rate: 2.5000e-04
Epoch 6/20
7108

In [63]:
results = model.evaluate(X_test, y_test, verbose=1)

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9995 - loss: 0.0036 - precision_2: 0.8864 - recall_2: 0.8105


In [64]:
def print_scores(y_true, y_pred, model_name):
    print(f"{model_name} Precision: {precision_score(y_true, y_pred):0.2f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred):0.2f}")
    print(f"{model_name} F1-Score: {f1_score(y_true, y_pred):0.2f}")

In [65]:
y_train_pred_ = model.predict(X_train)

7121/7121 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step


In [66]:
y_val_pred_ = model.predict(X_val)

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [80]:
# thresholds = [0.54,0.5 ,0.55,0.56, 0.6]
threshold = 0.5
y_val_pred = (y_val_pred_ > threshold).astype(int).flatten()
print(f'threshold: {threshold}')
print_scores(y_val, y_val_pred, 'Neural Network (Validation)')

threshold:0.5
Neural Network (Validation) Precision: 0.77
Neural Network (Validation) Recall: 0.77
Neural Network (Validation) F1-Score: 0.77


In [81]:
y_train_pred = (y_train_pred_ > threshold).astype(int).flatten()
print(f'threshold: {threshold}')
print_scores(y_train, y_train_pred, 'Neural Network (Train)')

threshold:0.5
Neural Network (Train) Precision: 0.85
Neural Network (Train) Recall: 0.85
Neural Network (Train) F1-Score: 0.85


In [69]:
y_pred_ = (model.predict(X_test))

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [83]:
print(f'threshold: {threshold}')
y_pred = (y_pred_ > threshold).astype("int32")
print_scores(y_test, y_pred, 'Neural Network (Test)')

threshold:0.5
Neural Network (Test) Precision: 0.91
Neural Network (Test) Recall: 0.80
Neural Network (Test) F1-Score: 0.85


In [84]:
model.save('neural_network_model_pca_80.keras')